# Set up the connection to MongoDB

In [1]:
from pymongo import MongoClient
from mongohandler import *
import requests
from IPython.display import display, HTML
from bson.son import SON
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# IMPORTANT : Remember to start mongodb server
# run $sudo systemctl start mongod
# Also, start the api.py Flask server
# $python3 api.py

Connected to MongoClient at: mongodb://localhost:27017


Hay que mover esta config a un config.py

In [2]:
flask_api = 'http://localhost:5007'    

DBURL = 'mongodb://localhost:27017'
client = MongoClient(DBURL)
print(f"Connected to MongoClient at: {DBURL}")
db = client.messenger

nltk.download('vader_lexicon')

Connected to MongoClient at: mongodb://localhost:27017


# Trying out our basic functions

To use the Messenger API, we will only use the python built-in `requests` module. 
These all the MongoDB functions been defined in the `mongohandler` module.

In [3]:
import pandas as pd
df = pd.read_csv('INPUT/RickAndMortyScripts.csv')
df.head(3)

### 1 - Create Users

In [5]:
usernames = [e for e in df.name.value_counts().index]
print(usernames[:5])

['Rick', 'Morty', 'Beth', 'Jerry', 'Summer']


In [6]:
for user in usernames:
    route = f'/user/create/{user}'
    res = requests.get(flask_api+route)
    
print(f'We have tried to create {len(usernames)} users.')
print("Last recorded event : ")    
display(HTML(res.text)) # Last user created    

We have tried to create 48 users.
Last recorded event : 


### 2 - Create Public Chats

In [7]:
chats_outline = df.groupby(['episode name', 'name']).agg({'line':'count'})
chats_outline

line
episode name                            name             
A Rickle in Time                        All Mortys     15
                                        All Ricks      32
                                        All Summers    13
                                        Beth           30
                                        Jerry          25
...                                                   ...
Vindicators 3  The Return of Worldender Morty          58
                                        Rick           80
                                        Summer          1
                                        Supernova      44
                                        Vance          17

[104 rows x 1 columns]

In [8]:
all_chat_titles = df['episode name'].drop_duplicates().values
all_chat_titles

array(['Pilot', 'Lawnmower Dog', 'Rick Potion 9', 'A Rickle in Time',
       'The Wedding Squanchers', 'Get Schwifty',
       'Interdimensional Cable 2  Tempting Fate',
       'The Rickshank Rickdemption', 'Pickle Rick',
       'Vindicators 3  The Return of Worldender',
       'Tales From the Citadel'], dtype=object)

In [37]:
for chat_title in all_chat_titles:
    participants = list(chats_outline.loc[chat_title].index) # These are the chat_titles and usernames
    route = f"/chat/create?title={chat_title}&users={participants}"
    res = requests.get(flask_api+route)
    
print('Example message from HTTP response:')
display(HTML(res.text))

Example message from HTTP response:


### 3 - Add New Users to an existing chat
To do this, I have create a new dummy user `rihp`, and added it to the `get_schwifty` chat.

In [10]:
username='rihp'
chat_title = "get_schwifty"

res = requests.get(flask_api+f'/user/create/{username}')
display(HTML(res.text))
print('------------------------------------------------------')
route = f"/chat/{chat_title}/adduser?&username={username}"
res = requests.get(flask_api+route)
display(HTML(res.text))

------------------------------------------------------


### 4 - Add Messages to a public chat

In [61]:
m_outline = df[['episode name', 'name', 'line']]
m_outline.head(3)

,episode name,name,line
0,Pilot,Rick,Morty! You gotta come on. Jus'... you gotta co...
1,Pilot,Morty,"What, Rick? What’s going on?"
2,Pilot,Rick,"I got a surprise for you, Morty."


### To send a message, the user must be part of the chat
This function is buggy and prints a lot of verbose.
Please fix 

`check_user_in_chat('rihp', 'pickle_rick')`

In [62]:
m_outline[m_outline['episode name'] == 'Pickle Rick'].tail(3)

,episode name,name,line
1476,Pickle Rick,Rick,Jaguar!
1477,Pickle Rick,Morty,"Who? Who was that, Rick?"
1478,Pickle Rick,Rick,"That, Morty, is why you don't go to therapy."


### This cell is not very efficient
it sends all the messages to the database

In [63]:
for i in m_outline.index:
    message = m_outline.iloc[i]
    
    chat_title = message['episode name']
    username = message['name']
    text = message['line']
    #print(username, text)
    route = f"/chat/{chat_title}/addmessage?username={username}&text={text}"
    res = requests.get(flask_api+route)
#display(HTML(res.text))

### 5 - Query all the messages sent to an specific chat
- (GET) `/chat/<chat_id>/list`

Aggregate the messages in the chat

In [64]:
cur = db.chat.aggregate([
    {
        '$lookup': {
            'from': 'messages', 
            'localField': 'messages', 
            'foreignField': '_id', 
            'as': 'messages'
        }
    }, {
        '$lookup': {
            'from': 'user', 
            'localField': 'participants', 
            'foreignField': '_id', 
            'as': 'participants'
        }
    }, {
        '$project': {
            '_id': 1, 
            'title': 1, 
            'participants': 1, 
            'messages': {
                'text': 1, 
                'username': 1
            }
        }
    }
])
query = list(cur)

### Access the specific chat and display some of the messages
This is not the specific chat, its only the first one in the list. fix this

In [65]:
for i in range(7):
    print(query[0]['messages'][i]['text'])

Morty! You gotta come on. Jus'... you gotta come with me.
What, Rick? What’s going on?
I got a surprise for you, Morty.
It's the middle of the night. What are you talking about?
Come on, I got a surprise for you.  Come on, hurry up.
Ow! Ow! You're tugging me too hard!
We gotta go, gotta get outta here, come on. Got a surprise for you Morty.


### 6 - Perform a Sentiment analysis on an specific chat message history
- (GET) `/chat/<chat_id>/sentiment`

In [66]:
def get_chat_doc(chat_title, query):
    for i in range(len(query)):
        if query[i]['title'] == chat_title:
            yield query[i]

In [79]:
episode = 'Get Schwifty'
chat_title = no_spaces(episode).lower()
chat_messages = list(get_chat_doc(chat_title, query))[0]['messages']

if len(chat_messages) != df['episode name'].value_counts()[episode]: 
    raise Exception('something is wrong here, the database has more messages than it should have')

In [80]:
sia = SentimentIntensityAnalyzer()

In [81]:
def analyze_chat_sentiment(chat_messages):
    for i in range(len(chat_messages)):
        text = chat_messages[i]['text']
        yield sia.polarity_scores(text) # Analyze this

In [82]:
sentiment = pd.DataFrame(list(analyze_chat_sentiment(chat_messages)))
display(sentiment.describe())
print(sentiment.describe().loc['mean'])

,neg,neu,pos,compound
count,221.000000,221.000000,221.000000,221.000000
mean,0.104633,0.780190,0.115186,0.046848
std,0.200661,0.239887,0.187185,0.433142
min,0.000000,0.000000,0.000000,-0.891000
25%,0.000000,0.647000,0.000000,-0.102700
50%,0.000000,0.809000,0.000000,0.000000
75%,0.133000,1.000000,0.191000,0.361200
max,1.000000,1.000000,1.000000,0.978500


neg         0.104633
neu         0.780190
pos         0.115186
compound    0.046848
Name: mean, dtype: float64


### 7 - Query all the messages sent from an specific user

### 8 - Recommender system takes in an `user_id` and return top-3 similar `users`
- (GET) `/user/<user_id>/recommend`

### 9 - Move local database to MongoAtlas cloud

### 10 - Prepare Docker Image

### 11  - Deploy Docker Image in Heroku

In [ ]:
# Some basic checking:
Is there a chat 